In [ ]:
import matplotlib.pyplot as plt
import scipy.constants as kn
import numpy as np
from sympy import symbols, cos, lambdify, diff, pi, factorial, exp
from scipy.constants import physical_constants as pkn 
from sympy.abc import n, x, a, r, theta, phi
from matplotlib import cm

In [ ]:
n = 4           # The principal quantum number 
l = 1           # The angular momentum quantum number
m = -1          # The magnetic momentum quantum number
theta = pi/4    # Elevation angle
phi = pi/4      # Azimuth angle

In [ ]:
# Definitions 
# Constants 
a0 = pkn['Bohr radius'][0] # Bohr radius


# Creating P_l(x)  corresponding to 
# Legendre Polynomials in the Rodrigues' form 
# Tested with page 47 of the Cambridge formulas book 
# It works
def Plx(l,x):
    return 1/(2**l*factorial(l))*diff((x**2-1)**l,x,l)


# Creating P_l^m(x) corresponding to Associated Legendre functions
# Tested with page 48 of the Cambridge formulas book # It works
def Plmx_possitive_part(l,m,x,theta):
    return ((-1)**m*(1-x**2)**(m/2)*diff(Plx(l,x),x,m)).subs(x,cos(theta))


# The criteria behind use a conditional is 
# because i must be simmetrical 
# Adding (-1) ** m in these equations is 
# the same as put it in the total wavefunction
def Plmx(l,m,x,theta):
    if m >= 0:
        return Plmx_possitive_part(l,m,x,theta)
    if m < 0:
        return (-1)**m*Plmx_possitive_part(l,abs(m),x,theta)


# Associated Laguerre polynomial formula ## Correct Issue #1
def Lab(n,l):
    aux_laguerre=[]
    for k in range(n-l):
        L=(factorial(n+l)*(-x)**k)/(factorial(2*l+1+k)*factorial(n-l-1-k)*factorial(k))
        aux_laguerre.append(L)
    return sum(aux_laguerre)


# Creating Y_l^m(theta,phi), corresponding to the 
# total angular contribution in the wavefunction
# Remember (-1)**m was remove to the assoc Legendre Poly, 
# this way here is innecesary
# Tested with page 49 of the Cambridge formulas book # It works
def Ylm_theta_phi(l,m,x,theta,phi):
    return ((2*l+1)/(4*pi)*factorial(l-abs(m))/factorial(l+abs(m)))**(1/2)*Plmx(l,m,x,theta)*exp(1j*m*phi)


# Creating R_{nl}(r), being the total 
# radial contribution in the wavefunction
# Tested using pages 49 and 96 and extracting the product 
# It works
def Rnlr(n,l,x):
    return (factorial(n-l-1)/(2*n*factorial(n+l)))**(1/2)*(2/(a*n))**(3/2)*x**l*exp(-x/2)*Lab(n,l)

 
# The next function correspond to the whole wavefunction 
# Tested with page 96 of the Cambridge formulas book # It works
def psi_nlm_theta_phi(n,l,m,x,theta,phi):
    return Rnlr(n,l,x).subs(x,2*r/(n*a))*Ylm_theta_phi(l,m,x,theta,phi)


def Psi(n,l,m):
    return psi_nlm_theta_phi(n,l,m,x,theta,phi)

In [ ]:
# Calculations
# The wavefunction evaluation with the three quantum numbers
psi = r*psi_nlm_theta_phi(n,l,m,x,theta,phi) 

# The complement of the wavefunction
psi_comp = r*psi_nlm_theta_phi(n,l,-m,x,theta,phi) 

# The orbital form # It will be an if statement
px = -1/2**0.5*(psi-psi_comp)

# The resolution of the plots in points quantity
res = 10000 

# Calculations and evaluations of the functions
psi = lambdify([r,a],psi,'numpy') 
px = lambdify([r,a],px, 'numpy') 

# The x-axis values
rx = np.linspace(0,1.75e-8,res)

# Function r*psi squared array eval
psi_squared = abs(psi(rx,a0))**2 

# Returns the max of the function
loc_max = int(np.where(psi_squared == max(psi_squared))[0])  

# returns the minimun value of the function forward the maximun
loc_min_follow = int(np.where(psi_squared[loc_max:] < 1e5)[0][11]) 
loc_min = loc_max+loc_min_follow

# Optimize functions limits
rx = np.linspace(0,rx[loc_min],res)
psi_squared = abs(psi(rx,a0))**2

In [ ]:
# The color variation in these plots
colors_psi = psi_squared/max(psi_squared)

plt.figure(figsize = [18,15])

# plt.subplot(212)
plt.scatter(rx,psi_squared,c = colors_psi,
            cmap = cm.viridis, marker = '.',
            label = '$|\psi_{%s %s %s}(\\theta,\phi)|^2$'%(n,l,m))

plt.ylim(0,max(psi_squared)*1.02)
plt.xlim(0,rx[-1])
plt.title('The squared wavefunction',size = 20)
plt.xlabel('$r$',size = 20)
plt.ylabel('$|\psi_{nlm}(\\theta,\phi)|^2$', size = 20)
plt.legend(loc = 'best', prop = {'size': 18})
plt.grid()

# figname = 'psi_'+str(n)+str(l)+str(m)+'.jpg'
# plt.savefig(figname, dpi = 1200)

In [ ]:
res = 1000
rx = np.linspace(0,rx[-1],res)
psi_squared = abs(psi(rx,a0))**2
upsiy = psi_squared*1 # Dulplicate the psi_y variable, so you can freely operate with this variable 
start = 100 #Determines minimun values
rx2 = np.array([]) # Create an empty array to fill it with the desired quatinties of rx_i
for k in range(start,res): # Choose start value equal 100 to reflect the nodal points of the wavefunction
    aux1 = np.where(upsiy == max(upsiy))[0][0] #, u.append(k) #Enable when u is active
    aux2 = np.ones(res+start-k)*rx[aux1]
    rx2 = np.append(rx2,aux2)
    upsiy[aux1] = 0

In [ ]:
# Next step: pair those r values with random angles to pair with
res2=100000 # Must be an integer
polar_psi_r=np.array([])
polar_psi_r=np.append(polar_psi_r,np.random.choice(rx2,res2))
polar_psi_angle=np.linspace(-np.pi,np.pi,res2)


In [ ]:
colors_psi=polar_psi_r/max(polar_psi_r)

plt.figure(figsize=[20,20]).add_subplot(projection='polar')

plt.scatter(polar_psi_angle,polar_psi_r,c=colors_psi,cmap=cm.viridis, marker='.')#, alpha=0.65)

# figname = 'psi_'+str(n)+str(l)+str(m)+'_radial_prob.jpg'
# plt.savefig(figname,dpi=1200)

In [ ]:
res = 10000
rx = np.linspace(0,8e-9,res)
Px = px(rx,a0)
Px2 = abs(Px)**2
colors_px=abs(Px)/max(abs(Px)) 

plt.figure(figsize=[18,15])

plt.subplot(221)
# Plot of px
plt.scatter(rx,np.real(Px),c=colors_px,cmap=cm.viridis, marker='.', label='Real part of: $P_x$')
plt.xlim(0,rx[-1])
plt.title('The wavefunction of hydrogen atom',size=20)
plt.xlabel('$r$',size=20)
plt.ylabel('$\psi_{nlm}(\\theta,\phi)$', size=20)
plt.legend(loc='best', prop={'size': 18})
plt.grid()

plt.subplot(222)
# Plot of px Imaginary part
plt.scatter(rx,np.imag(Px),c=colors_px,cmap=cm.viridis, marker='.', label='Imaginary part of: $P_x$')
plt.xlim(0,rx[-1])
plt.title('The wavefunction of hydrogen atom',size=20)
plt.xlabel('$r$',size=20)
plt.ylabel('$\psi_{nlm}(\\theta,\phi)$', size=20)
plt.legend(loc='best', prop={'size': 18})
plt.grid()

plt.subplot(212)
# Plot of px squared
plt.scatter(rx,np.real(Px2),c=colors_px,cmap=cm.viridis, marker='.', label='Whole wavefunction')
plt.xlim(0,rx[-1])
plt.title('The wavefunction of hydrogen atom',size=20)
plt.xlabel('$r$',size=20)
plt.ylabel('$\psi_{nlm}(\\theta,\phi)$', size=20)
plt.legend(loc='best', prop={'size': 18})
plt.grid()

In [ ]:
res = 1000
upx = Px*1 # Dulplicate the Px variable, so you can freely operate with this variable 
start = 100 #Determines minimun values
rx3 = np.array([]) # Create an empty array to fill it with the desired quatinties of rx
for k in range(start,res): # Choose start value equal 100 to reflect the nodal points of the wavefunction
    aux1 = np.where(upx == max(upx))[0][0] #, u.append(k) #Enable when u is active
    aux2 = np.ones(res+start-k)*rx[aux1]
    rx3 = np.append(rx2,aux2)
    upx[aux1] = 0


In [ ]:
res3 = 10000 # Must be an integer
polar_px_r = np.array([])
polar_px_r = np.append(polar_psi_r,np.random.choice(rx3,res3))
polar_px_angle = np.angle(polar_px_r)

len(polar_px_r), len(polar_px_angle)

In [ ]:
colors_px=polar_px_r/max(polar_px_r)
plt.figure(figsize=[20,20]).add_subplot(projection='polar')
plt.scatter(polar_px_angle,polar_px_r,c=colors_px,cmap=cm.viridis, marker='.')#, alpha=0.65)
#plt.title('Atom', size=20)
#plt.savefig('psi_71-1-cividis.jpg',dpi=300)